# Electron subshell configuration data

An electron shell is the set of allowed states that share the same principal quantum number $n$ that electrons may occupy. A subshell is the set of states defined by the azimuthal quantum number $l$ within an electron shell. The maximum number of electrons that can be held in a given subshell is equal to $4l + 2$.

All data related to a given subshell, such as the binding energy of an electron in the shell or the number of electrons in the subshell when the atom is neutral, is stored within an instance of `ElectronSubshellConfiguration` from the `dryad.atomic` subpackage.

## `ElectronSubshellConfiguration`

### Properties

`ElectronSubshellConfiguration` contains the data for a single electron subshell:
- the identifier of the subshell (`identifier`)
- the binding energy of the subshell (`binding_energy`)
- the number of neutrons of the subshell when the atom is neutral (`population`)
- the radiative transitions allowed for the subshell (`radiative_transitions`) which can be empty
- the non-radiative transitions allowed for the subshell (`non_radiative_transitions`) which can be empty

The `identifier` property is read-only while `binding_energy`, `population`, `radiative_transitions` and `non_radiative_transitions` are read and write properties.

`ElectronSubshellConfiguration` also provides a number of properties related to the transition data if any is defined for the subshell:
- the total number of allowed radiative transitions (`number_radiative_transitions`)
- the total number of allowed non-radiative transitions (`number_non_radiative_transitions`)
- the total number of allowed transitions (`number_transitions`), the sum of the number of radiative and non-radiative transitions
- a flag to indicate whether or not radiative transitions are available (`has_radiative_transitions`)
- a flag to indicate whether or not non-radiative transitions are available (`has_radiative_transitions`)
- a flag to indicate whether or not transitions are available (`has_transitions`)
- the total probability that a transition is radiative (`total_radiative_probability`), defined as the sum of the probability of each radiative transition
- the total probability that a transition is non-radiative (`total_non_radiative_probability`), defined as the sum of the probability of each non-radiative transition

### Construction

To create an instance of `ElectronSubshellConfiguration` the following arguments are required:
- `identifier`: the identifier of the subshell
- `binding_energy`: the binding energy of an electron for this shell
- `population`: the number of electrons in the subshll when the atom is in a neutron state

and the following arguments are optional:
- `radiative`: a list of `RadiativeTransitionData` instances (default is an empty list)
- `non_radiative`: a list of `NonRadiativeTransitionData` instances (default is an empty list)
- `normalise`: an option to indicate whether or not to normalise all probability data (default: no normalisation)

Note: the default behaviour in any `dryad` object that has probability data is that a user must explicitly request normalisation at construction time.

### Normalisation of transition probabilities

As indicated above, when constructing an instance of `ElectronSubshellConfiguration` the user has the possibility of normalising the probability data (the `normalise` option) so that the sum of all probabilities is 1. The `normalise()` allows a user to do the same after the `ElectronSubshellConfiguration` was created.

If the user changes one or more transition probabilities, this function can then be used to renormalise the data. It is the responsibility of the user to request this renormalisation whenever required.

### Example

In this example we will construct an instance of `ElectronSubshellConfiguration` for the K-subshell of Oxygen, using the contained in the ENDF/B-VIII.1 atomic relaxation library and do a few modifications of the transition energies. As provided, the transition energy values are not consistent with the binding energies of the other shells in the Oxygen relaxation data so they need to be recalculated. Since we require the binding energies of the other Oxygen subshells there is no built in method to do this (`AtomicRelaxation` which contains all the subshell data for an element does have such a capability).

The modifications are done in two different ways:
- the radiative transition data is updated as a hol
- the transition energies of the non-radiative transitions are updated by go over each transition

In addition to the updates to the transition energies, we also normalise the probability data for all transitions.

In [ ]:
from dryad.atomic import ElectronSubshellConfiguration
from dryad.atomic import RadiativeTransitionData
from dryad.atomic import NonRadiativeTransitionData
from dryad.id import ElectronSubshellID

def print_information( subshell ) :

    print( '  Subshell identifier             : {}'.format( subshell.identifier ) )
    print( '  Number radiative transitions    : {}'.format( subshell.number_radiative_transitions ) )
    print( '  Number non-radiative transitions: {}'.format( subshell.number_non_radiative_transitions ) )
    print( '  Total radiative probability     : {}'.format( subshell.total_radiative_probability ) )
    print( '  Total non-radiative probability : {}'.format( subshell.total_non_radiative_probability ) )
    print( '  Total probability               : {}'.format( subshell.total_radiative_probability +
                                                            subshell.total_non_radiative_probability ) )
    print( '' )
    print( '     Transition type   |   Probability   |  Energy' )
    print( '  ' + '- ' * 25 )
    for transition in subshell.radiative_transitions :

        print( '        Radiative      |  {:1.11f}  |  {:.2f}'.format( transition.probability, transition.energy ) )

    for transition in subshell.non_radiative_transitions :

        print( '      Non-radiative    |  {:1.11f}  |  {:.2f}'.format( transition.probability, transition.energy ) )

    print( '' )

# data for tke K subshell for oxygen, as given by ENDF/B-VIII.1
subshell_id = ElectronSubshellID( 'K' )
binding_energy = 538
number_electrons = 2
radiative_transitions = [ RadiativeTransitionData( ElectronSubshellID( 'L2' ), 0.00190768, 523.09 ),
                          RadiativeTransitionData( ElectronSubshellID( 'L3' ), 0.00380027, 523.13 ) ]
nonradiative_transitions = [ NonRadiativeTransitionData( ElectronSubshellID( 'L1' ),
                                                         ElectronSubshellID( 'L1' ),
                                                         0.178644 , 478.82 ),
                             NonRadiativeTransitionData( ElectronSubshellID( 'L1' ),
                                                         ElectronSubshellID( 'L2' ),
                                                         0.116224 , 493.86 ),
                             NonRadiativeTransitionData( ElectronSubshellID( 'L1' ),
                                                         ElectronSubshellID( 'L3' ),
                                                         0.230418 , 493.9 ),
                             NonRadiativeTransitionData( ElectronSubshellID( 'L2' ),
                                                         ElectronSubshellID( 'L2' ),
                                                         0.0110822, 508.9 ),
                             NonRadiativeTransitionData( ElectronSubshellID( 'L2' ),
                                                         ElectronSubshellID( 'L3' ),
                                                         0.291115 , 508.94 ),
                             NonRadiativeTransitionData( ElectronSubshellID( 'L3' ),
                                                         ElectronSubshellID( 'L3' ),
                                                         0.166809 , 508.98 ) ]

# create a ElectronSubshellConfiguration instance (not normalised)
subshell = ElectronSubshellConfiguration( id = subshell_id, energy = binding_energy,
                                          population = number_electrons,
                                          radiative = radiative_transitions,
                                          non_radiative = nonradiative_transitions )

print( 'Constructed subshell data:' )
print_information( subshell )

new_radiative = [ RadiativeTransitionData( ElectronSubshellID( 'L2' ), 0.00190768, 524.38 ),
                  RadiativeTransitionData( ElectronSubshellID( 'L3' ), 0.00380027, 524.38 ) ]

# change the radiative transition data as a whole
subshell.radiative_transitions = new_radiative

# change individual energy values in non-radiative transitions
subshell.non_radiative_transitions[0].energy = 481.04
subshell.non_radiative_transitions[1].energy = 495.9
subshell.non_radiative_transitions[2].energy = 495.9
subshell.non_radiative_transitions[3].energy = 510.76
subshell.non_radiative_transitions[4].energy = 510.76
subshell.non_radiative_transitions[5].energy = 510.76

# normalise the probabilities
subshell.normalise()

print( 'Updated and normalised subshell data:' )
print_information( subshell )

Constructed subshell data:
  Subshell identifier             : 1s1/2
  Number radiative transitions    : 2
  Number non-radiative transitions: 6
  Total radiative probability     : 0.00570795
  Total non-radiative probability : 0.9942922000000001
  Total probability               : 1.00000015

     Transition type   |   Probability   |  Energy
  - - - - - - - - - - - - - - - - - - - - - - - - - 
        Radiative      |  0.00190768000  |  523.09
        Radiative      |  0.00380027000  |  523.13
      Non-radiative    |  0.17864400000  |  478.82
      Non-radiative    |  0.11622400000  |  493.86
      Non-radiative    |  0.23041800000  |  493.90
      Non-radiative    |  0.01108220000  |  508.90
      Non-radiative    |  0.29111500000  |  508.94
      Non-radiative    |  0.16680900000  |  508.98

Updated and normalised subshell data:
  Subshell identifier             : 1s1/2
  Number radiative transitions    : 2
  Number non-radiative transitions: 6
  Total radiative probability     : 